In [1]:
import numpy as np

import math
import timeit
import argparse
from typing import Callable, Dict, Tuple

In [2]:
%load_ext Cython

In [3]:
# Number of trapezoids
N = 1000000

In [4]:
# Integration range
RANGE = {
    "sq": (0, 1),
    "sin": (0, math.pi),
    "exp": (0, 1),
    "inv": (1, 2)
}

In [5]:
def benchmark(trapz: Callable, f: Callable[[float], float], a: float, b: float, n: int) -> Tuple[float, float]:
    """
    Benchmark a trapz function on a given math function.

    Args:
        trapz (Callable): Integrating function
        f (Callable[[float], float]): Math function
        a (float): Lower bound for integration
        b (float): Upper bound for integration
        n (int): Number of trapezoids

    Returns:
        float: Area calculated from the trapezoidal rule
        float: Time per execution in seconds
    """

    # Find area first
    ar = impl(f, a, b, n)

    # Use timeit to benchmark
    timer = timeit.Timer(
        f"impl(f, {a}, {b}, {n})",
        globals={
            "impl": impl,
            "f": f
        }
    )

    exec_time = timer.autorange()
    t = exec_time[1] / exec_time[0] * 1e3

    return ar, t

In [6]:
def py_trapz(f: Callable[[float], float], a: float, b: float, n: int) -> float:
    """Pure Python Implementation - Loop from a to b and add trapezoidal area"""
    pointer = a
    step_size = (b - a ) / n
    area = 0

    # Needed to check if we reached the upper limit
    # Inaccuracies in floating points may cause the check to fail
    TOL = step_size * 0.01

    # Store current f(x) and f(x + step)
    pointer_f = f(pointer)
    pointer_plus_f = f(pointer + step_size)

    # Loop from a to b and add area
    while b - pointer > TOL:
        area += (pointer_f + pointer_plus_f)
        pointer += step_size
        pointer_f = pointer_plus_f
        pointer_plus_f = f(pointer + step_size)

    # We multiply by step_size / 2 in the end to avoid having to do it on each loop
    return area * step_size / 2

def np_trapz_raw_py_f(f: Callable[[float], float], a: float, b: float, n: int) -> float:
    """Numpy implementation of the trapezoidal rule"""
    
    x = np.linspace(a, b, n + 1)
    # f(x) is not necessarily a function that works on arrays - we need to vectorise it
    y = np.vectorize(f)(x)
    return np.trapz(y, x, (b - a) / n)

In [7]:
%%cython --annotate

# cython: infer_types=True
# cython: cdivision=True

import cython

cpdef double c_trapz_raw(f, double a, double b, int n):
    """Raw Cython implementation: Same algorithm as in pure Python implementation."""
    cdef double pointer   = a
    cdef double step_size = (b - pointer) / n
    cdef double area      = 0
    cdef double TOL       = step_size * 0.1
    
    cdef double pointer_f      = f(pointer)
    cdef double pointer_plus_f = f(pointer + step_size)

    while b - pointer > TOL:
        area += (pointer_f + pointer_plus_f)
        pointer += step_size
        pointer_f = pointer_plus_f
        pointer_plus_f = f(pointer + step_size)

    return area * step_size / 2

# Raw Python Implementations of Math Functions

In [8]:
# Implementations of the trapezoidal rule
IMPLEMENTATIONS = {
    "np": np_trapz_raw_py_f,
    "py": py_trapz,
    "c": c_trapz_raw
}

# Math functions to test it on
FUNCTIONS = {
    "sq": math.sqrt,
    "sin": math.sin,
    "exp": math.exp,
    "inv": lambda x: 1 / x
}

for f_name, func in FUNCTIONS.items():
    print(f"\nBenchmarking for function {f_name}():")
    for impl_name, impl in IMPLEMENTATIONS.items():
        # Run benchmark
        ar, t = benchmark(impl, func, *RANGE[f_name], N)
        print(f"Implementation: {impl_name}\t Time: {t:10.3f} ms\t Area: {ar:10.5f}")


Benchmarking for function sq():
Implementation: np	 Time:    211.331 ms	 Area:    0.66667
Implementation: py	 Time:    142.082 ms	 Area:    0.66667
Implementation: c	 Time:     41.020 ms	 Area:    0.66667

Benchmarking for function sin():
Implementation: np	 Time:    202.812 ms	 Area:    2.00000
Implementation: py	 Time:    126.980 ms	 Area:    2.00000
Implementation: c	 Time:     41.243 ms	 Area:    2.00000

Benchmarking for function exp():
Implementation: np	 Time:    147.733 ms	 Area:    1.71828
Implementation: py	 Time:    136.555 ms	 Area:    1.71828
Implementation: c	 Time:     50.883 ms	 Area:    1.71828

Benchmarking for function inv():
Implementation: np	 Time:    389.296 ms	 Area:    0.69315
Implementation: py	 Time:    162.598 ms	 Area:    0.69315
Implementation: c	 Time:     64.403 ms	 Area:    0.69315


# NumPy Implementations of the Math Functions:

In [9]:
def np_trapz(f: Callable[[float], float], a: float, b: float, n: int) -> float:
    """Better NumPy implementation: We implicitly assume that f is vectorised"""
    x = np.linspace(a, b, n + 1)
    return np.trapz(f(x), x, (b - a) / n)

In [10]:
# Implementations of the trapezoidal rule
IMPLEMENTATIONS = {
    "np": np_trapz,
    "py": py_trapz,
    "c": c_trapz_raw
}

# Mathematical functions - we use NumPy's implementations
# So that NumPy can take full advantage of the built in C optimizations
FUNCTIONS = {
    "sq": np.sqrt,
    "sin": np.sin,
    "exp": np.exp,
    "inv": np.reciprocal
}

for f_name, func in FUNCTIONS.items():
    print(f"\nBenchmarking for function {f_name}():")
    for impl_name, impl in IMPLEMENTATIONS.items():
        # Benchmarking
        ar, t = benchmark(impl, func, *RANGE[f_name], N)
        print(f"Implementation: {impl_name}\t Time: {t:10.3f} ms\t Area: {ar:10.5f}")


Benchmarking for function sq():
Implementation: np	 Time:     96.555 ms	 Area:    0.66667
Implementation: py	 Time:    855.073 ms	 Area:    0.66667
Implementation: c	 Time:    770.970 ms	 Area:    0.66667

Benchmarking for function sin():
Implementation: np	 Time:     84.233 ms	 Area:    2.00000
Implementation: py	 Time:    964.090 ms	 Area:    2.00000
Implementation: c	 Time:    770.972 ms	 Area:    2.00000

Benchmarking for function exp():
Implementation: np	 Time:     80.549 ms	 Area:    1.71828
Implementation: py	 Time:    847.563 ms	 Area:    1.71828
Implementation: c	 Time:    643.418 ms	 Area:    1.71828

Benchmarking for function inv():
Implementation: np	 Time:     41.391 ms	 Area:    0.69315
Implementation: py	 Time:    952.099 ms	 Area:    0.69315
Implementation: c	 Time:    682.926 ms	 Area:    0.69315


# C Implementations for Math Functions

In [11]:
%%cython --annotate

# cython: cdivision=True

from libc.math cimport sin, exp, sqrt

c_sin = sin
c_exp = exp
c_sqrt = sqrt

cpdef double c_reciprocal(double x):
    return 1 / x

In [12]:
# Trapezoidal rule implementations
IMPLEMENTATIONS = {
    "np": np_trapz,
    "py": py_trapz,
    "c": c_trapz_raw
}

# Functions for each mathematical func
# Each implementation - np, py and c has a different function
# This is useful because I can tailor each function to the implementation
FUNCTIONS = {
    "sq": {
        "np": np.sqrt,
        "py": math.sqrt,
        "c": c_sqrt
    },
    "sin": {
        "np": np.sin,
        "py": math.sin,
        "c": c_sin
    },
    "exp": {
        "np": np.exp,
        "py": math.exp,
        "c": c_exp
    },
    "inv": {
        "np": np.reciprocal,
        "py": lambda x: 1 / x,
        "c": c_reciprocal
    }
}

for f_name, func_dict in FUNCTIONS.items():    
    print(f"\nBenchmarking for function {f_name}():")
    for impl_name, impl in IMPLEMENTATIONS.items():
        # Benchmarking
        ar, t = benchmark(impl, func_dict[impl_name], *RANGE[f_name], N)
        print(f"Implementation: {impl_name}\t Time: {t:10.3f} ms\t Area: {ar:10.5f}")


Benchmarking for function sq():
Implementation: np	 Time:     30.251 ms	 Area:    0.66667
Implementation: py	 Time:    130.066 ms	 Area:    0.66667
Implementation: c	 Time:     32.791 ms	 Area:    0.66667

Benchmarking for function sin():
Implementation: np	 Time:     25.994 ms	 Area:    2.00000
Implementation: py	 Time:    124.777 ms	 Area:    2.00000
Implementation: c	 Time:     38.217 ms	 Area:    2.00000

Benchmarking for function exp():
Implementation: np	 Time:    119.860 ms	 Area:    1.71828
Implementation: py	 Time:    188.262 ms	 Area:    1.71828
Implementation: c	 Time:     36.459 ms	 Area:    1.71828

Benchmarking for function inv():
Implementation: np	 Time:     15.540 ms	 Area:    0.69315
Implementation: py	 Time:    162.427 ms	 Area:    0.69315
Implementation: c	 Time:     27.684 ms	 Area:    0.69315


# Removing Python Overhead in Function Call

In [13]:
%%cython --annotate

# cython: infer_types=True
# cython: cdivision=True

from libc.math cimport sin, exp, sqrt

# Define each math function as a C class
# Specify that they do not require the GIL
# Removes the need for Python interpreter to handle the function call

cdef class Function:
    cdef double evaluate(self, double x) noexcept nogil:
        return 0

cdef class c_sqrt_function(Function):
    cdef double evaluate(self, double x) noexcept nogil:
        return sqrt(x)

cdef class c_sin_function(Function):
    cdef double evaluate(self, double x) noexcept nogil:
        return sin(x)

cdef class c_exp_function(Function):
    cdef double evaluate(self, double x) noexcept nogil:
        return exp(x)

cdef class c_inv_function(Function):
    cdef double evaluate(self, double x) noexcept nogil:
        return 1 / x

# Now we specify f  astype Function
# Instead of a Python Callable
cpdef double c_trapz(Function f, double a, double b, int n):
    cdef double step_size = (b - a) / n
    cdef double area      = 0

    cdef int i

    for i in range(0, n):
        area += (f.evaluate(a + i * step_size) + f.evaluate(a + (i + 1) * step_size))

    return area * step_size / 2

In [14]:
# Trapezoidal rule implementations
IMPLEMENTATIONS = {
    "np": np_trapz,
    "py": py_trapz,
    "c": c_trapz
}

# Math functions for each implementation
FUNCTIONS = {
    "sq": {
        "np": np.sqrt,
        "py": math.sqrt,
        "c": c_sqrt_function()
    },
    "sin": {
        "np": np.sin,
        "py": math.sin,
        "c": c_sin_function()
    },
    "exp": {
        "np": np.exp,
        "py": math.exp,
        "c": c_exp_function()
    },
    "inv": {
        "np": np.reciprocal,
        "py": lambda x: 1 / x,
        "c": c_inv_function()
    }
}

for f_name, func_dict in FUNCTIONS.items():
    print(f"\nBenchmarking for function {f_name}():")
    for impl_name, impl in IMPLEMENTATIONS.items():
        # Benchmarking
        ar, t = benchmark(impl, func_dict[impl_name], *RANGE[f_name], N)
        print(f"Implementation: {impl_name}\t Time: {t:10.3f} ms\t Area: {ar:10.5f}")


Benchmarking for function sq():
Implementation: np	 Time:     76.300 ms	 Area:    0.66667
Implementation: py	 Time:    152.434 ms	 Area:    0.66667
Implementation: c	 Time:      5.505 ms	 Area:    0.66667

Benchmarking for function sin():
Implementation: np	 Time:    101.199 ms	 Area:    2.00000
Implementation: py	 Time:    124.676 ms	 Area:    2.00000
Implementation: c	 Time:     21.638 ms	 Area:    2.00000

Benchmarking for function exp():
Implementation: np	 Time:     83.511 ms	 Area:    1.71828
Implementation: py	 Time:    136.864 ms	 Area:    1.71828
Implementation: c	 Time:     16.479 ms	 Area:    1.71828

Benchmarking for function inv():
Implementation: np	 Time:     16.554 ms	 Area:    0.69315
Implementation: py	 Time:    231.902 ms	 Area:    0.69315
Implementation: c	 Time:      5.065 ms	 Area:    0.69315


# Parallelism

In [15]:
%%cython --annotate

# cython: infer_types=True
# cython: cdivision=True

from cython.parallel cimport prange

from libc.math cimport sin, exp, sqrt

# Define each math function as a C class
# Specify that they do not require the GIL
# Removes the need for Python interpreter to handle the function call

cdef class Function:
    cdef double evaluate(self, double x) noexcept nogil:
        return 0

cdef class c_sqrt_function(Function):
    cdef double evaluate(self, double x) noexcept nogil:
        return sqrt(x)

cdef class c_sin_function(Function):
    cdef double evaluate(self, double x) noexcept nogil:
        return sin(x)

cdef class c_exp_function(Function):
    cdef double evaluate(self, double x) noexcept nogil:
        return exp(x)

cdef class c_inv_function(Function):
    cdef double evaluate(self, double x) noexcept nogil:
        return 1 / x

# Now we specify f  astype Function
# Instead of a Python Callable
cpdef double c_trapz_parallel(Function f, double a, double b, int n):
    cdef double step_size = (b - a) / n
    cdef double area      = 0

    cdef int i

    # Parallel range
    for i in prange(0, n, nogil=True):
        area += (f.evaluate(a + i * step_size) + f.evaluate(a + (i + 1) * step_size))

    return area * step_size / 2

In [16]:
# Trapezoidal rule implementations
IMPLEMENTATIONS = {
    "np": np_trapz,
    "py": py_trapz,
    "c": c_trapz_parallel
}

# Math functions for each implementation
FUNCTIONS = {
    "sq": {
        "np": np.sqrt,
        "py": math.sqrt,
        "c": c_sqrt_function()
    },
    "sin": {
        "np": np.sin,
        "py": math.sin,
        "c": c_sin_function()
    },
    "exp": {
        "np": np.exp,
        "py": math.exp,
        "c": c_exp_function()
    },
    "inv": {
        "np": np.reciprocal,
        "py": lambda x: 1 / x,
        "c": c_inv_function()
    }
}

for f_name, func_dict in FUNCTIONS.items():
    print(f"\nBenchmarking for function {f_name}():")
    for impl_name, impl in IMPLEMENTATIONS.items():
        # Benchmarking
        ar, t = benchmark(impl, func_dict[impl_name], *RANGE[f_name], N)
        print(f"Implementation: {impl_name}\t Time: {t:10.3f} ms\t Area: {ar:10.5f}")


Benchmarking for function sq():
Implementation: np	 Time:     70.669 ms	 Area:    0.66667
Implementation: py	 Time:    173.729 ms	 Area:    0.66667
Implementation: c	 Time:      6.472 ms	 Area:    0.66667

Benchmarking for function sin():
Implementation: np	 Time:     22.913 ms	 Area:    2.00000
Implementation: py	 Time:    124.673 ms	 Area:    2.00000
Implementation: c	 Time:     18.459 ms	 Area:    2.00000

Benchmarking for function exp():
Implementation: np	 Time:     24.400 ms	 Area:    1.71828
Implementation: py	 Time:    135.281 ms	 Area:    1.71828
Implementation: c	 Time:     15.904 ms	 Area:    1.71828

Benchmarking for function inv():
Implementation: np	 Time:     20.011 ms	 Area:    0.69315
Implementation: py	 Time:    163.186 ms	 Area:    0.69315
Implementation: c	 Time:      6.868 ms	 Area:    0.69315


# Benchmarking on f(x) = x^2

In [17]:
%%cython --annotate

# cython: infer_types=True
# cython: cdivision=True

import cython
from cython.parallel cimport prange

from libc.math cimport sin, exp, sqrt

# Define each math function as a C class
# Specify that they do not require the GIL
# Removes the need for Python interpreter to handle the function call

cdef class c_sq_function:
    cdef double evaluate(self, double x) noexcept nogil:
        return x * x

# Now we specify f as type Function
# Instead of a Python Callable
cpdef double c_trapz_sq(c_sq_function f, double a, double b, int n):
    cdef double step_size = (b - a) / n
    cdef double area      = 0

    cdef int i

    for i in range(0, n):
        area += (f.evaluate(a + i * step_size) + f.evaluate(a + (i + 1) * step_size))

    return area * step_size / 2

In [18]:
# Benchmarking on f(x) = x^2
# From x = 0 to 10, with N = 10^7

benchmarks = [
    ("np", np_trapz, np.square),
    ("py", py_trapz, lambda x: x * x),
    ("c", c_trapz_sq, c_sq_function())
]

print("Benchmarking for f(x) = x^2")
for impl_name, impl, f_x in benchmarks:
    ar, t = benchmark(impl, f_x, 0, 10, 10**7)
    print(f"Implementation: {impl_name}\t Time: {t:10.3f} ms\t Area: {ar:10.5f}")

Benchmarking for f(x) = x^2
Implementation: np	 Time:    243.239 ms	 Area:  333.33333
Implementation: py	 Time:   1526.559 ms	 Area:  333.33333
Implementation: c	 Time:     50.314 ms	 Area:  333.33333
